In [83]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import pickle
import json
import os

In [84]:
path = "/Users/iremn/PythonClass/Spiced/gradient_garlic-code_Work/Week_04/Day_1"

In [85]:
artist = input("Enter your artist: ") 
artist_name = str(artist).lower()

Enter your artist:  queen


In [86]:
artist_name_re = re.sub(' ','-',artist_name)

In [87]:
# Create target Directory if don't exist
if not os.path.exists(artist_name_re):
    os.mkdir(artist_name_re)
    print("Directory " , artist_name_re ,  " Created ")
else:    
    print("Directory " , artist_name_re ,  " already exists")

Directory  queen  already exists


In [88]:
artist_name_ht = artist_name_re + '-lyrics.html'

In [89]:
#Build http for metrolyrics
wbpg = 'https://www.metrolyrics.com/'+artist_name_ht

In [90]:
song_links = requests.get(wbpg)
parsed = BeautifulSoup(song_links.text, 'html.parser')


In [91]:
http_range = []
for i in range(len(parsed.find_all(attrs={'class': 'pages'})[0].find_all('a'))):
    http_range.append(parsed.find_all(attrs={'class': 'pages'})[0].find_all('a')[i].get('href'))
    

In [92]:
http_range

['https://www.metrolyrics.com/queen-alpage-1.html',
 'https://www.metrolyrics.com/queen-alpage-2.html',
 'https://www.metrolyrics.com/queen-alpage-3.html',
 'https://www.metrolyrics.com/queen-alpage-4.html',
 'https://www.metrolyrics.com/queen-alpage-5.html']

In [93]:
#Collect all song https
song_http = []
for address in http_range:
    songs_pg = requests.get(address)
    pg = BeautifulSoup(songs_pg.text, 'html.parser')
    titles = pg.find_all(attrs = {'class': 'page-content tabs-wrapper clearfix'})[0].find_all(attrs = {'class':'title'})
    for i in range(len(titles)):
        song_http.append(titles[i].get('href'))
    

In [94]:
for link in song_http:
    pj = requests.get(link)
    parser = BeautifulSoup(pj.text, 'html.parser')
    song_name = link.split('/')[3].split('.htm')[0]
    lyrics = parser.find_all(attrs = {'id':'lyrics-body-text'})[0]
    verse = lyrics.find_all(attrs = {'class' : 'verse'})
    lyrics_text = []
    for i in range(len(verse)):
        a = verse[i].get_text()
        lyrics_text.append(a)
        #re.sub(r'\n', '\s', lyrics_text) # to replace
    with open(f'./{artist_name_re}/{song_name}.txt', 'w') as f:
        json.dump(lyrics_text, f)
